# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
weather_data = pd.read_csv("weather_data2.csv")

weather_data.head()


,city,lat,lon,max temp,humidity,cloudiness,wind speed,date
0,George Town,5.41,100.34,84.20,88,20,3.36,1591750597
1,Cape Town,-33.93,18.42,59.00,87,75,23.04,1591750597
2,Hithadhoo,-0.60,73.08,83.53,72,100,8.08,1591750597
3,Hobart,-42.88,147.33,52.00,53,75,9.17,1591750500
4,Chui,-33.70,-53.46,52.03,94,0,2.37,1591750597


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
# Configure gmaps with API key

gmaps.configure(api_key=g_key)

In [6]:
# Store Lat and Lon into locations

locations = weather_data[["lat", "lon"]].astype(float)

humidity = weather_data["humidity"].astype(float)

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                dissipating=False, max_intensity=100,
                                point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [22]:
ideal_temp = weather_data[(weather_data['max temp']>70) & (weather_data['max temp'] <80)]
ideal_temp_wind = ideal_temp[ideal_temp['wind speed']<10]
ideal_weather = ideal_temp_wind[ideal_temp_wind['cloudiness']==0]

ideal_weather.head()

,city,lat,lon,max temp,humidity,cloudiness,wind speed,date
9,Port Alfred,-33.59,26.89,73.00,36,0,3.00,1591750598
66,Itarema,-2.92,-39.92,77.23,86,0,8.77,1591750606
106,Turbat,26.00,63.04,78.58,89,0,1.50,1591750611
112,Albany,42.60,-73.97,73.99,49,0,4.00,1591750612
154,Jardim,-21.48,-56.14,73.74,78,0,3.31,1591750617


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [30]:
hotel_df = pd.DataFrame({"City":ideal_weather['city'],
                        "Lat":ideal_weather['lat'],
                        "Lon":ideal_weather['lon']})

hotel_df


,City,Lat,Lon
9,Port Alfred,-33.59,26.89
66,Itarema,-2.92,-39.92
106,Turbat,26.00,63.04
112,Albany,42.60,-73.97
154,Jardim,-21.48,-56.14
206,Mozarlândia,-14.74,-50.57
247,Carnarvon,-24.87,113.63
267,Rāvar,31.27,56.81
305,Rosário Oeste,-14.84,-56.43
308,Caravelas,-17.71,-39.25


In [35]:
hotel_df["Hotel Name"] = ""
hotel_df["Hotel Address"] = ""
hotel_df

,City,Lat,Lon,Hotel Name,Hotel Address
9,Port Alfred,-33.59,26.89,,
66,Itarema,-2.92,-39.92,,
106,Turbat,26.00,63.04,,
112,Albany,42.60,-73.97,,
154,Jardim,-21.48,-56.14,,
206,Mozarlândia,-14.74,-50.57,,
247,Carnarvon,-24.87,113.63,,
267,Rāvar,31.27,56.81,,
305,Rosário Oeste,-14.84,-56.43,,
308,Caravelas,-17.71,-39.25,,


In [40]:
# create a params dict that will be updated with new city each iteration

params = {
    "radius": 5000,
    "types": "lodging",
    "keyword": "hotel",
    "key": g_key    
}

# use lat/lng to identify hotels

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lon"]
    
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    name_address = requests.get(base_url, params=params)
    
    name_address = name_address.json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
        hotel_df.loc[index, "Hotel Address"] = name_address["results"][0]["vicinity"]
    
    except (KeyError, IndexError):
        print("Missing field/result...skipping.")
        



Missing field/result...skipping.
Missing field/result...skipping.
Missing field/result...skipping.
Missing field/result...skipping.


In [39]:
hotel_df.head()

,City,Lat,Lon,Hotel Name,Hotel Address
9,Port Alfred,-33.59,26.89,The Halyards Hotel,"1 Albany Road, Port Alfred"
66,Itarema,-2.92,-39.92,Hotel do David,Itarema
106,Turbat,26.00,63.04,Shapur Hotel,"Main Rd, Turbat"
112,Albany,42.60,-73.97,,
154,Jardim,-21.48,-56.14,Hotel Estância,"R. Duque de Caxias, 2041 - Centro, Guia Lopes ..."


In [44]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations2 = hotel_df[["Lat", "Lon"]]

In [47]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations2)

fig.add_layer(markers)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))